In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pre = pd.read_excel('EP47_45degree_FFP.xlsx')
next = pd.read_excel('EP47_45degree_PR.xlsx',sheet_name='Test Result')

In [3]:
pre.sample(3)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
49,17812,Epix_Pro_ProdScan_Pack,77,Double check SN from barcode,2/383,0.52
54,17822,Epix_Pro_ALS,6,ALS Luminance (WLED OFF),2/438,0.46
48,17812,Epix_Pro_ProdScan_Pack,54,Check Wifi Country Code,2/383,0.52


In [4]:
next.sample(5)

,ItemName Type,Process Type,Item,ItemName,Count/Count ESN,Retest%
190,18547,Epix_Pro_MultiUpdater,26,Check Boot Block,8/739,1.08
100,17803,Epix_Pro_FT2,50,外部電流(反插)|VEXT Current(Reverse) BL&OHR On,4/619,0.65
219,17798,Epix_Pro_AutoATE,34,Keypad Back,3/738,0.41
176,17800,Epix_Pro_CT,74,Galileo,1/656,0.15
75,12434,Watch_ClickTest,38,Button4_Stroke,1/91,1.10


# Tweak and Filter

Blacklist

In [5]:
ProcessTypeBlackList = 'Pack|Click|RabbitCard|EasyCard|DeleteBundle|ProdScan|FileCopyer'
ItemNameBlackList = 'ESN|Check ProductionMap|Fixture ID'

tweak_filter def

In [6]:
def tweak(raw):
    return(raw
    .drop('Comment',axis=1, errors='ignore')
    .rename(columns = lambda name : name.replace(" ",'').replace('/','').replace('%',''))
    .sort_values('Retest', ascending=False)
    .astype({'ItemNameType':'category','ProcessType':'category','Item':'category','ItemName':'category'})
    .assign(CountESN = lambda df : df.CountCountESN.str.split('/').str[1].astype('int16'))
    .query("~ProcessType.str.contains(@ProcessTypeBlackList) and ~ItemName.str.contains(@ItemNameBlackList)")
    )
def tweak_filter(raw):
    return(raw
    .pipe(tweak)
    .query("CountESN > CountESN.quantile(0.05)")
    .query("Retest >= Retest.mean()")
    )

In [7]:
# tweak_filter(next).to_excel('tweak_EP47_45degree_PR.xlsx', index=False)

# Merge two phases

In [8]:
pre_next = pd.merge(
    left=tweak_filter(pre), right=tweak(next), how='left',
    left_on=['ItemNameType','ProcessType','Item','ItemName'], right_on=['ItemNameType','ProcessType','Item','ItemName'],
    suffixes=['_pre','_next'])

In [17]:
# pre_next.to_excel('output.xlsx', index=False)

# RetryImprove def

In [9]:
def RetryImprove(df):
    return((100 * (df.Retest_pre.sum() - df.Retest_next.sum()) / df.Retest_pre.sum()).round(3))

In [10]:
RetryImprove(pre_next)

-40.191

In [11]:
def RetryDiff(df):
    return(df
     .assign(diff = df.Retest_pre - df.Retest_next)
     [['ProcessType','ItemName','diff']]
     .sort_values('diff', ascending=False)
     .dropna(axis=0)
     .style.bar('diff', color=['red','green'])
     )
RetryDiff(pre_next)

,ProcessType,ItemName,diff
2,Epix_Pro_FT2,ANT_power 2450MHz BY REF (sapphire),5.940000
16,Epix_Pro_FT1,006J Comparison,0.880000
21,Epix_Pro_Baro,1 ATM diff with golden,0.760000
15,Epix_Pro_FT2,BLE LESC test|藍芽LESC測試,0.460000
20,Epix_pro_RTESN,光學心跳模組檢查|Check OHRM LED,0.210000
13,Epix_Pro_ECG_ATE,Motor Resonant Frequency,-0.020000
7,Epix_Pro_FT2,Manifest Event Test,-1.950000
0,Epix_Pro_ECG_FT,ECG impedance short,-2.270000
19,Epix_Pro_ATE_TI,AFE Power Consumption data,-6.330000
10,Epix_Pro_FT1,電池電壓|Battery Voltage,-7.230000


# Total improvement rate

In [12]:
100*((tweak(pre).Retest.sum() - tweak(next).Retest.sum())/ tweak(pre).Retest.sum())

-221.22246977754872